---------------------------------

# **Digital Image Processing & Computer Vision Project**

# Traffic Light Detection & Recognition

**Prepared By: Sourabh Vijay Jadhav (SE21MEEE007)**

----------------------------------

# Step 1: Importing the necessary libraries

1) **OS**: The OS module in Python provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc.

2) **cv2**: OpenCV (Open Source Computer Vision Library) is an open-source computer vision and machine learning software library. OpenCV is a huge open-source library for computer vision, machine learning, and image processing.

3) **numpy**: is a Python library used for working with arrays. It also has functions for working in domain of linear algebra, fourier transform, and matrices.

In [4]:
import os
import cv2
import numpy as np

----------------------------

# Step 2: Defining the function to detect & recognize the traffic lights

HSV (Hue, Saturation, Value) color space: It stores color information in a cylindrical representation of RGB color points. It attempts to depict the colors as perceived by the human eye. Hue value varies from 0-179, Saturation value varies from 0-255 and Value value varies from 0-255. It is mostly used for color segmentation purpose.

-----------------------------------------------------------------------------------------------

**Line 47-50** - hough circle detect function

**cv.HoughCircles(image, method, dp, minDist[, circles[, param1[, param2[, minRadius[, maxRadius]]]]]	) ->circles**


**image**	8-bit, single-channel, grayscale input image.


**circles**	Output vector of found circles.

**method**	Detection method.

**dp**	Inverse ratio of the accumulator resolution to the image resolution. For example, if dp=1 , the accumulator has the same resolution as the input image.

**minDist**	Minimum distance between the centers of the detected circles. If the parameter is too small, multiple neighbor circles may be falsely detected in addition to a true one. If it is too large, some circles may be missed.

**param1**	First method-specific parameter.

**param2**	Second method-specific parameter.

**minRadius**	Minimum circle radius.

**maxRadius**	Maximum circle radius.

In [5]:
def detect(filepath, file):

    # putText() method is used to draw a text string on any image.
    font = cv2.FONT_HERSHEY_SIMPLEX

    # reading the image from directly file path
    img = cv2.imread(filepath+file)

    # assigning the cimg variable to above image
    # cimg variable is used to save the detected results
    cimg = img

    # as cv2 reads the image into BGR format, below line converts the BGR image to HSV line
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # color range
    # red color range
    lower_red1 = np.array([0,100,100])
    upper_red1 = np.array([10,255,255])
    lower_red2 = np.array([160,100,100])
    upper_red2 = np.array([180,255,255])

    # green color range
    lower_green = np.array([50,50,50])
    upper_green = np.array([90,255,255])

    # yellow color range
    # lower_yellow = np.array([15,100,100])
    # upper_yellow = np.array([35,255,255])
    lower_yellow = np.array([15,150,150])
    upper_yellow = np.array([35,255,255])

    # mask for red 
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    maskr = cv2.add(mask1, mask2)
    #cv2.imshow("Red Mask", maskr)

    # mask for green
    maskg = cv2.inRange(hsv, lower_green, upper_green)
    #cv2.imshow("Green Mask", maskg)

    # mask for yellow
    masky = cv2.inRange(hsv, lower_yellow, upper_yellow)
    #cv2.imshow("Yellow Mask", masky)

    size = img.shape
    #print('size of the image: ', size)

    # hough circle detect
    r_circles = cv2.HoughCircles(maskr, cv2.HOUGH_GRADIENT, 1, 80, param1=50, param2=10, minRadius=0, maxRadius=30)
    g_circles = cv2.HoughCircles(maskg, cv2.HOUGH_GRADIENT, 1, 60, param1=50, param2=10, minRadius=0, maxRadius=30)
    y_circles = cv2.HoughCircles(masky, cv2.HOUGH_GRADIENT, 1, 30, param1=50, param2=5, minRadius=0, maxRadius=30)

    # traffic light detect
    r = 5
    bound = 4.0 / 10
    if r_circles is not None:

        # np.around: This mathematical function helps user to evenly 
        # round array elements to the given number of decimals.

        # np.uint16: is a type object representing the type of array 
        # scalars of uint16 dtype.
        r_circles = np.uint16(np.around(r_circles))

        for i in r_circles[0, :]:
            if i[0] > size[1] or i[1] > size[0]or i[1] > size[0]*bound:
                continue

            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):

                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += maskr[i[1]+m, i[0]+n]
                    s += 1
            if h / s > 50:
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(maskr, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'STOP',(i[0], i[1]), font, 1,(255,0,0),2,cv2.LINE_AA)

    if g_circles is not None:
        g_circles = np.uint16(np.around(g_circles))

        for i in g_circles[0, :]:
            if i[0] > size[1] or i[1] > size[0] or i[1] > size[0]*bound:
                continue

            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):

                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += maskg[i[1]+m, i[0]+n]
                    s += 1
            if h / s > 100:
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(maskg, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'GO',(i[0], i[1]), font, 1,(255,0,0),2,cv2.LINE_AA)

    if y_circles is not None:
        y_circles = np.uint16(np.around(y_circles))

        for i in y_circles[0, :]:
            if i[0] > size[1] or i[1] > size[0] or i[1] > size[0]*bound:
                continue

            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):

                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += masky[i[1]+m, i[0]+n]
                    s += 1
            if h / s > 50:
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(masky, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'WARN',(i[0], i[1]), font, 1,(255,0,0),2,cv2.LINE_AA)

    cv2.imshow('detected results', cimg)
    cv2.imwrite(path+'//result//'+file, cimg)
    # cv2.imshow('maskr', maskr)
    # cv2.imshow('maskg', maskg)
    # cv2.imshow('masky', masky)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

---------------------------------------------------------

# Step 3: Detection & recognition of traffic lights from different scenarios

In [6]:
if __name__ == '__main__':

    path = os.path.abspath('..')+'\\TrafficLight-Detector-master\\light\\'
    for f in os.listdir(path):
        print (f)
        if f.endswith('.jpg') or f.endswith('.JPG') or f.endswith('.png') or f.endswith('.PNG'):
            # Resizing the Image
            detect(path, f)

1.jpg
10.jpg
11.jpg
15.jpg
17.jpg
18.jpg
2.jpg
21.jpg
22.jpg
23.jpg
24.jpg
4.jpg
5.jpg
6.jpg
7.jpg
IMG_0000.jpg
IMG_0227.JPG
IMG_0229.JPG
IMG_0234.JPG
IMG_0236.JPG
IMG_0239.JPG
IMG_0240.JPG
IMG_0254.JPG
IMG_0273.JPG
IMG_0285.JPG
IMG_0316.jpg
IMG_0358.jpg


--------------------------------------------